In [1]:
import numpy as np
import pandas as pd
import h5py
import os
import skimage.io as sio
import tqdm

/home/shubham/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
data_root = "/home/shubham/datasets/subset/ShapeNet/"
dataset_dir = "/home/shubham/datasets/"

os.makedirs(os.path.join(dataset_dir, "hdf_data"), exist_ok=True)
save_path = os.path.join(dataset_dir, "hdf_data")

In [3]:
os.makedirs(os.path.join(dataset_dir, "hdf_data"), exist_ok=True)
save_path = os.path.join(dataset_dir, "hdf_data")

In [4]:
def save_dict_to_hdf5(dic, filename):
    """
    ....
    """
    with h5py.File(filename, 'w') as h5file:
        recursively_save_dict_contents_to_group(h5file, '/', dic)

def recursively_save_dict_contents_to_group(h5file, path, dic):
    """
    ....
    """
    for key, item in dic.items():
        if isinstance(item, (np.ndarray, np.int64, np.float64, str, bytes)):
            h5file[path + key] = item
        elif isinstance(item, dict):
            recursively_save_dict_contents_to_group(h5file, path + key + '/', item)
        else:
            raise ValueError('Cannot save %s type'%type(item))

def load_dict_from_hdf5(filename):
    """
    ....
    """
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')

def recursively_load_dict_contents_from_group(h5file, path):
    """
    ....
    """
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item.value
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
    return ans

In [5]:
def load_data(path):   
    # Load the pointcloud.npz and points.npz file
    pc_file = np.load(os.path.join(path, "pointcloud.npz"))
    points_file = np.load(os.path.join(path, "points.npz"))
    
    # create image placeholder and camera data placeholder
    img_data = []
    cam_data = None
    
    # Load images
    for imx in os.listdir(os.path.join(path, "img_choy2016")):
        current = os.path.join(path, "img_choy2016", imx)
        if 'npz' in imx:
            cam_data = np.load(current)
        else:
            img_current = sio.imread(current)
            if img_current.ndim == 2:
                img_current = np.stack([img_current, img_current, img_current], axis=-1)
            img_data.append(img_current)
    img_data = np.asarray(img_data)
    
    all_data = {
        'images': img_data,
        'camera': dict(cam_data),
        'points': dict(points_file),
        'pointcloud': dict(pc_file)
    }
    
    return all_data

In [6]:
for cid in os.listdir(data_root):
    objs_path = os.path.join(data_root, cid)
    obj_list = os.listdir(objs_path)
    for obx in tqdm.tqdm_notebook(obj_list):
        current_path = os.path.join(objs_path, obx)
        new_filename = "{}_{}.h5".format(cid, obx)
        
        try:
            data_current = load_data(current_path)
            save_dict_to_hdf5(data_current, os.path.join(save_path, new_filename))
        except:
            print("Error at {}-{}".format(cid, obx))

<ipython-input-6-35487b7e2f39>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for obx in tqdm.tqdm_notebook(obj_list):


  0%|          | 0/289 [00:00<?, ?it/s]

Error at 02933112-test.lst


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/277 [00:00<?, ?it/s]

...

  0%|          | 0/292 [00:00<?, ?it/s]

Error at 02828884-val.lst


  0%|          | 0/275 [00:00<?, ?it/s]

Error at 04401088-val.lst


  0%|          | 0/307 [00:00<?, ?it/s]